# Downloads data from APPLAUSE  

##  WORK IN PROGRESS

This script automatically downloads from the APPLAUSE archive, all data necessay to run the analysis scripts on a plate sequence.

For each entry (plate ID) in a sequence, we need  to download two kinds of data: plate scans, and source tables. Analysis requires two scans and four source tables per pair of exposures:

- plate scan 1 
- plate scan 2
- sources 1
- sources calib 1
- sources 2
- sources calib 2

Data is downloaded only when not already physically present in the data directory (DATADIR path in file *settings.py*).

In [1]:
import os
from pathlib import Path
import shutil
from importlib import reload
from urllib.parse import urlsplit
import requests
import json

import pyvo as vo

from astropy.io import fits
from astropy.utils.data import download_file
from astropy.table import Table

from applause_token import token
from settings import DATAPATH, get_parameters, get_table_sources, current_dataset, images, fname

## Functions

In [2]:
def download_scan(table, plate_id, test=False):

    # get URL for this plate ID 
    mask = table['plate_id'] == plate_id
    table_1 = table[mask]
    url_1 = table_1['filename_scan'][0]
    
    # get file name from URL
    parsed_url = urlsplit(url_1)
    filename = parsed_url.path.split('/')[-1]
    
    # if file already exists in data storage, bail out
    file_path = Path(os.path.join(DATAPATH, filename))
    if file_path.is_file():
        return

    # Download the file and get the local filename/path
    local_file_path = download_file(url_1)
    if test:
        print(f"File downloaded to: {local_file_path}")

    # Move and rename file into data directory
    new_path = shutil.move(local_file_path, os.path.join(DATAPATH, filename))
    if test:
        print("new path is: ", new_path)

    # TEST: open the local FITS file using the obtained path
    if test:
        with fits.open(new_path) as hdul:
            # Access the header information or data
            header_info = hdul[0].header
            print(f"TEST: telescope used: {header_info.get('TELESCOP')}")
            print(f"TEST: date: {header_info.get('DATE-AVG')}")

    # once image is successfully downloaded, we need to update the 'images.json' 
    # dictionary with the new association "plate id: file name" entry.
    images[str(plate_id)] = filename        
            
    try:
        json_file = open('images.json', 'w')
        json.dump(images, json_file, indent=4)
    except IOError as e:
        print(f"Error writing to file: {e}")

In [8]:
def download_sources_table(plate_id, calib=False):
    '''
    This function requires that a 'token' variable be present in the namespace.

    The usual way to do this is to create a 1-line .py import file named 
    'applause_token'. The file will define the variable with the token definition 
    you got from APPLAUSE (look for the 'API token' entry in the dropdown menu 
    from your login name in the APPLAUSE web page). Something like:
    
    token = '2a2223453fdf3eq89aweafs6da05415hhh1d4369193e'
    
    '''
    # If table already exists in data storage, bail out.
    # We check just one table of the set of four, assuming 
    # that it is an error if one or more of the four tables
    # are missing.
    table_name = fname(get_table_sources(plate_id, calib=calib))
    file_path = Path(table_name)
    if file_path.is_file():
        return

    # setup
    name = 'APPLAUSE',
    url = 'https://www.plate-archive.org/tap'
    ap_token = 'Token ' + token

    # assemble query string
    qstr = 'SELECT * FROM applause_dr4.source'
    if calib:
        qstr = qstr + '_calib'
    qstr = qstr + ' WHERE plate_id = ' + str(plate_id) + ' ORDER BY source_id'

#     print('\npyvo version %s \n' % vo.__version__)
#     print('TAP service %s \n' % name)

    # session mechanics
    tap_session = requests.Session()
    tap_session.headers['Authorization'] = ap_token

    tap_service = vo.dal.TAPService(url, session=tap_session)

    lang = 'PostgreSQL'

    job = tap_service.submit_job(qstr, language=lang)
    job.run()

    job.wait(phases=["COMPLETED", "ERROR", "ABORTED"], timeout=70.)

    job.raise_if_error()
    results = job.fetch_result()

    # return an astropy Table instance
    return results.to_table()

In [4]:
par = get_parameters(current_dataset)
par

{'nproc': 8,
 'sextractor_flags': 4,
 'model_prediction': 0.8,
 'max_flux_threshold': 0.3,
 'elongation': 1.5,
 'annular_bin': 7,
 'flag_rim': 0,
 'min_acceptable_flux': 20000,
 'min_fwhm': 4.0,
 'max_fwhm': 7.5,
 'qfit_max': 5.0,
 'cfit_max': 5.0,
 'invert_east': [False, False],
 'invert_north': [False, False],
 'table1': 'sources_9319.csv',
 'table2': 'sources_9320.csv',
 'table1_calib': 'sources_calib_9319.csv',
 'table2_calib': 'sources_calib_9320.csv',
 'table_matched': 'table_match_9319_9320.fits',
 'table_non_matched': 'table_nomatch_9319_9320.fits',
 'table_psf_nonmatched': 'table_psf_nomatch_9319_9320.fits',
 'image1': 'GS00768_x.fits',
 'image2': 'GS00769_x.fits'}

## Downloading plate scans

In [5]:
catalog_applause = Table.read('footprints_5.csv', format='ascii.csv')

plate_id = 9536

In [6]:
download_scan(catalog_applause, plate_id, test=True)

## Downloading source tables

In [9]:
results = download_sources_table(9537)

results

source_id,process_id,scan_id,plate_id,archive_id,source_num,x_source,y_source,a_source,b_source,theta_source,erra_source,errb_source,errtheta_source,elongation,x_peak,y_peak,flag_usepsf,x_image,y_image,erra_image,errb_image,errtheta_image,x_psf,y_psf,erra_psf,errb_psf,errtheta_psf,mag_auto,magerr_auto,flux_auto,fluxerr_auto,mag_iso,magerr_iso,flux_iso,fluxerr_iso,flux_max,flux_radius,isoarea,sqrt_isoarea,background,sextractor_flags,dist_center,dist_edge,annular_bin,flag_rim,flag_negradius,flag_clean,model_prediction,timestamp_insert,timestamp_update
,,,,,,,,pix,pix,deg,pix,pix,deg,,,,,,,pix,pix,deg,,,pix,pix,deg,mag,mag,,,mag,mag,,,,pix,pix2,pix,,,pix,pix,,,,,,Time[yyyy-mm-dd hh:mm:ss],Time[yyyy-mm-dd hh:mm:ss]
int64,int32,int32,int32,int32,int32,float64,float64,float32,float32,float32,float32,float32,float32,float32,int32,int32,int16,float64,float64,float32,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,float32,float32,int16,float64,float64,int16,int16,int16,int16,float32,str32,str32
40351400000001,35140,12411,9537,102,1,13678.21,12.706304046100698,0.95269793,0.7483539,1.7484757,0.061299905,0.048481155,-0.03601748,1.273058,13678,12,0,13678.21,11.976801,0.061299905,0.048481155,-0.03601748,--,--,--,--,--,12.812965,0.04847577,74957.266,3345.8665,13.226682,0.042406466,51206.766,1999.5377,7135.061,1.5030704,10,3.1622777,3.5561812,0,11576.6455,11.976801,9,1,0,0,0.7851216,2022-06-13 05:08:21.644377+00,2022-06-13 05:08:21.644377+00
40351400000002,35140,12411,9537,102,2,5405.011,11.19289612080857,1.0557806,0.66475004,-63.02059,0.07041305,0.04309491,-63.453293,1.588237,5405,10,0,5405.011,10.463127,0.07041305,0.04309491,-63.453293,--,--,--,--,--,12.770935,0.044065967,77915.91,3161.5466,13.402248,0.049849257,43561.29,1999.5377,5373.817,1.7072415,10,3.1622777,2.6599889,0,12806.579,10.463127,9,1,0,0,1.4747726e-08,2022-06-13 05:08:21.644377+00,2022-06-13 05:08:21.644377+00
40351400000003,35140,12411,9537,102,3,7339.8057,5.442055276579616,2.880487,1.6609885,31.589947,0.042834096,0.02580029,27.683685,1.7342005,7338,4,0,7339.8057,4.711276,0.042834096,0.02580029,27.683685,--,--,--,--,--,10.833819,0.018426936,463951.0,7872.1875,10.84182,0.012826353,460545.0,5439.3306,13085.2,3.046696,65,8.062258,3.1866975,24,12036.196,4.711276,9,1,0,0,3.855104e-18,2022-06-13 05:08:21.644377+00,2022-06-13 05:08:21.644377+00
40351400000004,35140,12411,9537,102,4,7323.8403,23.69068730975487,1.7659911,1.3846135,-52.0868,0.050473094,0.0401464,-50.264473,1.2754397,7323,23,0,7323.8403,22.963114,0.050473094,0.0401464,-50.264473,--,--,--,--,--,11.7305,0.03277355,203142.06,6130.4663,11.912216,0.023641763,171835.77,3740.7925,7740.641,2.3732853,33,5.7445626,3.7924163,0,12024.329,22.963114,9,1,0,0,2.2879782e-05,2022-06-13 05:08:21.644377+00,2022-06-13 05:08:21.644377+00
40351400000005,35140,12411,9537,102,5,13618.624,32.48732737176686,1.2978839,0.7408572,-26.941662,0.07002929,0.040018506,-25.76533,1.7518679,13618,32,0,13618.624,31.7613,0.07002929,0.040018506,-25.76533,--,--,--,--,--,12.499794,0.04998215,100018.98,4603.2812,12.9377985,0.037055306,66816.03,2279.8237,7561.286,1.8502262,12,3.4641016,-14.659882,0,11545.419,31.7613,9,1,0,0,1.0212587e-08,2022-06-13 05:08:21.644377+00,2022-06-13 05:08:21.644377+00
40351400000006,35140,12411,9537,102,6,7316.0,51.730262010014286,1.6879027,0.59610474,85.96965,0.03633499,0.012242562,86.25617,2.831554,7316,51,0,7316.0,51.007614,0.03633499,0.012242562,86.25617,--,--,--,--,--,12.029995,0.036458157,154170.8,5175.675,12.340514,0.019663528,115822.87,2097.1328,12376.68,1.4735128,11,3.3166249,383.93048,3,12000.543,51.007614,9,1,0,0,3.01472e-17,2022-06-13 05:08:21.644377+00,2022-06-13 05:08:21.644377+00
40351400000007,35140,12411,9537,102,7,7301.623,56.80511896460839,2.1732643,1.5692949,0.77968675,0.03948553,0.027950766,0.87146795,1.3848667,7302,56,0,7301.623,56.083363,0.03948553,0.027950766,0.87146795,--,--,--,--,--,11.23777,0.039358355,319810.06,1159

In [10]:
results = download_sources_table(9537, calib=True)

results

source_id,process_id,scan_id,plate_id,archive_id,solution_num,annular_bin,dist_center,dist_edge,sextractor_flags,model_prediction,ra_icrs,dec_icrs,ra_error,dec_error,gal_lon,gal_lat,ecl_lon,ecl_lat,x_sphere,y_sphere,z_sphere,healpix256,healpix1024,nn_dist,zenith_angle,airmass,natmag,natmag_error,bpmag,bpmag_error,rpmag,rpmag_error,natmag_plate,natmag_correction,natmag_residual,phot_range_flags,phot_calib_flags,color_term,cat_natmag,match_radius,gaiaedr3_id,gaiaedr3_gmag,gaiaedr3_bp_rp,gaiaedr3_dist,gaiaedr3_neighbors,timestamp_insert,timestamp_update,pos
,,,,,,,px,px,,,,,arcsec,arcsec,,,,,,,,,,arcsec,deg,,mag,,,,,,mag,mag,mag,,,,mag,arcsec,,,,arcsec,,Time[yyyy-mm-dd hh:mm:ss],Time[yyyy-mm-dd hh:mm:ss],"(radians,radians)"
int64,int32,int32,int32,int32,int16,int16,float64,float64,int16,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,int32,int32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,float32,float32,float32,int64,float32,float32,float32,int32,str32,str32,object
40351400000001,35140,12411,9537,102,0,9,11576.6455,11.976801,0,0.7851216,123.733039914166,62.03941765860208,0.100272,0.0970944,154.61054935244636,33.465717170888766,110.17027132165461,40.94041661276047,-0.26037147894187884,0.38992326784214976,0.8832703652613496,123947,1983153,57.214954,51.68741,1.6093227,--,--,--,--,--,--,0.0,--,--,0,0,--,--,0.0,--,--,--,--,0,2022-06-13 20:56:05.830206+00,2022-06-13 20:56:05.830206+00,"(2.15954894000376 , 1.08279210416252)"
40351400000002,35140,12411,9537,102,0,9,12806.579,10.463127,0,1.4747726e-08,124.26798679330372,64.12019492042548,0.100272,0.0970944,152.0992771929063,33.50776295832169,109.6436491737398,42.998292291394584,-0.24576898923355955,0.36071663921975144,0.8997116816631437,124105,1985688,128.4161,50.091167,1.5554633,--,--,--,--,--,--,0.0,--,--,0,0,--,--,0.0,--,--,--,--,0,2022-06-13 20:56:05.830206+00,2022-06-13 20:56:05.830206+00,"(2.16888552436798 , 1.11910851838197)"
40351400000003,35140,12411,9537,102,0,9,12036.196,4.711276,24,3.855104e-18,124.1387894351412,63.633715535029935,0.100272,0.0970944,152.68666050035745,33.50375781036071,109.77157982384892,42.5175954572296,-0.24923318534666203,0.3675795908872747,0.895973249424948,124055,1984890,22.072102,50.46119,1.5675187,--,--,--,--,--,--,0.0,--,--,0,0,--,--,0.0,--,--,--,--,0,2022-06-13 20:56:05.830206+00,2022-06-13 20:56:05.830206+00,"(2.16663060508317 , 1.11061785136374)"
40351400000004,35140,12411,9537,102,0,9,12024.329,22.963114,0,2.2879782e-05,124.12953823339143,63.638266747328615,0.100272,0.0970944,152.68174535455327,33.49919794790892,109.764398199468,42.52068927424981,-0.24913389917883882,0.3675609142852367,0.8960085237148969,124055,1984890,20.335224,50.461132,1.5675169,--,--,--,--,--,--,0.0,--,--,0,0,--,--,0.0,--,--,--,--,0,2022-06-13 20:56:05.830206+00,2022-06-13 20:56:05.830206+00,"(2.16646914115287 , 1.11069728500331)"
40351400000005,35140,12411,9537,102,0,9,11545.419,31.7613,0,1.0212587e-08,123.72598877648997,62.054963224296436,0.100272,0.0970944,154.59228889028748,33.46118646670216,110.1600696928771,40.954316574325844,-0.26019042233430045,0.3897559757737717,0.883397545544549,123947,1983155,57.214954,51.67904,1.6090274,--,--,--,--,--,--,0.0,--,--,0,0,--,--,0.0,--,--,--,--,0,2022-06-13 20:56:05.830206+00,2022-06-13 20:56:05.830206+00,"(2.1594258744353 , 1.08306342546797)"
40351400000006,35140,12411,9537,102,0,9,12000.543,51.007614,3,3.01472e-17,124.11419876051845,63.64106221968716,0.100272,0.0970944,152.67926069184955,33.492133212732305,109.754442693007,42.521292185126754,-0.24901096685728266,0.3675914061060917,0.8960301872937918,124055,1984890,8.553467,50.464256,1.5676197,--,--,--,--,--,--,0.0,--,--,0,0,--,--,0.0,--,--,--,--,0,2022-06-13 20:56:05.830206+00,2022-06-13 20:56:05.830206+00,"(2.16620141684571 , 1.11074607520011)"
40351400000007,35140,12411,9537,102,0,9,12000.608,56.083363,0,0.9999995,124.11229056882712,63.64482591903832,0.100272,0.0970944,152